## Creo las tablas de alumnos y generos

In [40]:
from peewee import MySQLDatabase, AutoField, CharField, DateField, ForeignKeyField, Model, IntegerField, DateTimeField, Field, JOIN, fn
from playhouse.hybrid import hybrid_property
import datetime
import tb_alumnos, tb_gender, tb_alumnos_genero

In [41]:
db = MySQLDatabase('tp', user='root', password='$Pumita7', host='localhost', port=3306)

In [42]:
## Ver si puedo crear una clase personas y que profesores y alumnos la hereden

class Alumnos(Model): 
    first_name = CharField()
    last_name = CharField()
    email = CharField(unique=True)
    birthdate = DateField()
    personal_id = IntegerField()
    insert_date= DateTimeField(default=datetime.datetime.now())
    update_date=DateTimeField()


    class Meta:
        database = db 

    @hybrid_property
    def age(self):
        today = datetime.date.today()
        age = today.year - self.birthdate.year
        if (today.year, today.month, today.day) < (today.year, self.birthdate.month, self.birthdate.day):
            age -= 1
        return age



class Gender(Model):
    gender = CharField() 

    class Meta:
        database = db 



class AlumnosGenero(Model):
	alumno = ForeignKeyField(Alumnos, backref="alumnosxgenero")
	gender = ForeignKeyField(Gender, backref="alumnosxgenero") 
	
	class Meta:
		database = db


In [43]:
db.connect() #me conecto
db.create_tables([Alumnos, Gender, AlumnosGenero])   ### agregar un control de que si existe la tabla no la vuelva a crear ??


In [44]:
dataset_alumnos = tb_alumnos.alum2
dataset_alumnos

[{'first_name': 'Darcy',
  'last_name': 'Beat',
  'email': 'dbeat1@youtube.com',
  'birthdate': Timestamp('2000-01-30 00:00:00'),
  'personal_id': 96878816},
 {'first_name': 'Jenda',
  'last_name': 'Daniels',
  'email': 'jdaniels2@tinyurl.com',
  'birthdate': Timestamp('1989-06-26 00:00:00'),
  'personal_id': 4276697},
 {'first_name': 'Corbet',
  'last_name': 'Lampet',
  'email': 'clampet3@nih.gov',
  'birthdate': Timestamp('2001-02-09 00:00:00'),
  'personal_id': 17602181},
 {'first_name': 'Seumas',
  'last_name': 'Bisacre',
  'email': 'sbisacre4@ft.com',
  'birthdate': Timestamp('1993-09-17 00:00:00'),
  'personal_id': 47200626},
 {'first_name': 'Rab',
  'last_name': 'Bury',
  'email': 'rbury5@over-blog.com',
  'birthdate': Timestamp('2000-07-23 00:00:00'),
  'personal_id': 37548402},
 {'first_name': 'Joli',
  'last_name': 'Tonbridge',
  'email': 'jtonbridge6@jiathis.com',
  'birthdate': Timestamp('1998-06-17 00:00:00'),
  'personal_id': 39622051},
 {'first_name': 'Marie-ann',
  'las

In [45]:
dataset_gender = tb_gender.gender2
dataset_gender

[{'gender': 'Female'},
 {'gender': 'Non-binary'},
 {'gender': 'Male'},
 {'gender': 'Polygender'},
 {'gender': 'Genderqueer'},
 {'gender': 'Agender'},
 {'gender': 'Bigender'},
 {'gender': 'Genderfluid'}]

In [46]:
with db.atomic():
   Alumnos.insert_many(dataset_alumnos).execute()
   Gender.insert_many(dataset_gender).execute()
   

In [50]:
dataset_alumnos_genero = tb_alumnos_genero.alumnos_genero
print(type(dataset_alumnos_genero))
len(dataset_alumnos_genero)
dataset_alumnos_genero

<class 'list'>


[[99873643, 'Female'],
 [96878816, 'Female'],
 [4276697, 'Non-binary'],
 [17602181, 'Male'],
 [47200626, 'Male'],
 [37548402, 'Male'],
 [39622051, 'Female'],
 [38509143, 'Polygender'],
 [321480, 'Female'],
 [95960451, 'Male'],
 [89777835, 'Male'],
 [78496135, 'Female'],
 [31466063, 'Female'],
 [6464620, 'Male'],
 [94880036, 'Female'],
 [27665809, 'Female'],
 [57664399, 'Male'],
 [67969140, 'Female'],
 [21168785, 'Male'],
 [93584168, 'Male'],
 [58117562, 'Male'],
 [24712296, 'Male'],
 [61899399, 'Male'],
 [67475949, 'Female'],
 [97200696, 'Male'],
 [11441971, 'Male'],
 [58608642, 'Female'],
 [59794180, 'Female'],
 [54976366, 'Male'],
 [7516520, 'Male'],
 [5373008, 'Male'],
 [67092865, 'Female'],
 [5772263, 'Genderqueer'],
 [420795, 'Male'],
 [98168154, 'Male'],
 [19351247, 'Male'],
 [8574833, 'Female'],
 [16715043, 'Female'],
 [66346512, 'Female'],
 [12662873, 'Female'],
 [41736870, 'Male'],
 [6821144, 'Male'],
 [61329381, 'Male'],
 [33590813, 'Female'],
 [43403620, 'Male'],
 [62116894,

In [48]:

for pers_id, genero in dataset_alumnos_genero:
	alumno = Alumnos.get(Alumnos.personal_id == pers_id)
	gen = Gender.get(Gender.gender == genero)
	AlumnosGenero.create(alumno = alumno, gender=gen )

AlumnosDoesNotExist: <Model: Alumnos> instance matching query does not exist:
SQL: SELECT `t1`.`id`, `t1`.`first_name`, `t1`.`last_name`, `t1`.`email`, `t1`.`birthdate`, `t1`.`personal_id`, `t1`.`insert_date`, `t1`.`update_date` FROM `alumnos` AS `t1` WHERE (`t1`.`personal_id` = %s) LIMIT %s OFFSET %s
Params: [99873643, 1, 0]